In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import scale
from sklearn.metrics import roc_auc_score
from sklearn.grid_search import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
import time
import sys
reload(sys)
sys.setdefaultencoding("utf-8")

In [2]:
train_fname = '../data/train_con_1.csv'
test_fname = '../data/test_con_1.csv'
y_fname = '../data/y_train_1.csv'
df_y = pd.read_csv(y_fname, sep=';')
df = pd.read_csv(train_fname, sep=';')
df_test = pd.read_csv(test_fname, sep=';')
f_con = ['APP_NB','APP_NB_PAYS','APP_NB_TYPE','NB_CLASSES','NB_ROOT_CLASSES','NB_SECTORS','NB_FIELDS','INV_NB',
        'INV_NB_PAYS','INV_NB_TYPE','cited_n','cited_nmiss','cited_age_min','cited_age_median','cited_age_max','cited_age_mean',
        'cited_age_std','NB_BACKWARD_NPL','NB_BACKWARD_XY','NB_BACKWARD_I','NB_BACKWARD_AUTRE','NB_BACKWARD_PL',
        'NB_BACKWARD','pct_NB_IPC','pct_NB_IPC_LY','oecd_NB_ROOT_CLASSES','oecd_NB_BACKWARD_PL','oecd_NB_BACKWARD_NPL',
        'IDX_ORIGIN','IDX_RADIC','PRIORITY_MONTH','FILING_MONTH','PUBLICATION_MONTH','BEGIN_MONTH']
(n_samples,n_variables) = (df.shape[0],len(f_con))

X_train = df[f_con].values
y_train = df_y['VARIABLE_CIBLE'].values == 'GRANTED'
X_test = df_test[f_con].values
print(len(f_con))

In [ ]:
n_threads = -1
clf = KNeighborsClassifier(n_neighbors=20, weights='uniform', algorithm='auto',
                           leaf_size=30, p=2, metric='minkowski', metric_params=None)
clf.fit(X_train,y_train)

In [ ]:
n_taken = 2000
start = time.time()
y_pred_train = clf.predict_proba(X_train[0:n_taken])[:, 1]
print('{} secondes'.format(time.time() - start))
print('Score (optimiste) sur le train : %s' % roc_auc_score(y_train[0:n_taken], y_pred_train))
print(clf.get_params())

In [ ]:
n_taken = n_samples
nb_thread = 4
clf = KNeighborsClassifier(weights='uniform', algorithm='auto',
                            p=2, metric='minkowski', metric_params=None)
parameters = {'n_neighbors':[300,310,320,330,340,350,360,370,380,390,400],'leaf_size':[2,3,4]}
def scorer(estimator, X, y):
    return roc_auc_score(y, estimator.predict_proba(X)[:,1])
gscv = GridSearchCV(clf, parameters, scorer, n_jobs = nb_thread)
start = time.time()
gscv.fit(X_train[0:n_taken],y_train[0:n_taken])
print(time.time() - start)
y_pred_train = gscv.predict_proba(X_train)[:, 1]
print('Score (optimiste) sur le train : {}'.format(gscv.best_score_))
print(gscv.best_params_)

In [ ]:
# 0.610172093545 400 2
# 0.610130745681 350 3 (la soumission obtient 0.615250228154, donc clairement on overfit pas.)
# 0.610127300706 500 3
# 0.609805696513 300 3

In [ ]:
pd.DataFrame(data=y_pred_train, columns=['VARIABLE_CIBLE']).to_csv(path_or_buf='../y_trained/cont_KNN_3_24.02.26.txt', sep=';')

In [ ]:
from sklearn.externals import joblib
joblib.dump(clf, '../trained_models/cont_KNN_2_24_02_16.pkl')

In [ ]:
y_test = gscv.predict_proba(X_test)[:, 1]

In [ ]:
np.savetxt('../subs/continue_KNN3.txt', y_test, fmt='%s')

#### La première sub avec k = 5 a bien overfit (résultat de 5.55 environ). Il faudrait faire une jolie CV mais là, ça risque vraiment de prendre du temps. On peut donc essayer avec de plus grandes valeurs de k pour l'instant et voir ce qu'il se passe.
#### La deuxième sub avec k = 20 a encore beaucoup trop overfit...

### On va tenter d'autres classifieurs basés sur KNN.

In [3]:
from sklearn.ensemble import RandomForestClassifier
n_taken = n_samples
clf = RandomForestClassifier(max_depth=8)
start=time.time()
clf.fit(X_train[0:n_taken], y_train[0:n_taken])
print(time.time() - start)
start=time.time()
y_pred_train = clf.predict_proba(X_train)[:, 1]
print(time.time() - start)
print('Score (optimiste) sur le train : %s' % roc_auc_score(y_train, y_pred_train))

In [4]:
n_taken = n_taken
nb_thread = 4
clf = RandomForestClassifier()
#'max_leaf_nodes':[28,30,32,34,36,38,40,42,44,46,48,50,52,54,56,58,60]
parameters = {'oob_score':[True,False],'max_depth':[9,10,11],'min_samples_leaf':[1,2,3],'min_weight_fraction_leaf':[0,0.2],'min_samples_split':[2,3,4]}
def scorer(estimator, X, y):
    return roc_auc_score(y, estimator.predict_proba(X)[:,1])
gscv = GridSearchCV(clf, parameters, scorer, n_jobs = nb_thread)
start = time.time()
gscv.fit(X_train[0:n_taken],y_train[0:n_taken])
print(time.time() - start)
print('Score réaliste sur le train : {}'.format(gscv.best_score_))
# y_pred_valid = gscv.predict_proba(X_train[n_taken:])[:, 1]
# print('Score sur la validation : {}'.format(roc_auc_score(y_train[n_taken:], y_pred_valid)))
print(gscv.best_params_)

In [ ]:
# 0.612881909952 {'min_samples_split': 1, 'max_leaf_nodes': 26, 'max_depth': 8}
# 0.617207656745 {'min_samples_split': 2, 'max_leaf_nodes': 58, 'min_weight_fraction_leaf': 0, 'max_depth': 9, 'min_samples_leaf': 1}
# sub à 0.551656973888... Très gros overfit. D'où vient-il ?
# 0.621387050444 {'oob_score': True, 'max_leaf_nodes': 58, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0, 'min_samples_split': 2, 'max_depth': 8}
# 0.632137935136 {'min_samples_split': 3, 'oob_score': True, 'min_weight_fraction_leaf': 0, 'max_depth': 10, 'min_samples_leaf': 2}
# On obtient un score équivalent avec une validation.
# Soumission à 0.635146915779, tout va bien.
# 0.633689374595 {'min_samples_split': 4, 'oob_score': True, 'min_weight_fraction_leaf': 0, 'max_depth': 11, 'min_samples_leaf': 3}

In [ ]:
y_pred_train = gscv.predict_proba(X_train)[:, 1]

In [ ]:
plt.hist(y_pred_valid)

In [ ]:
pd.DataFrame(data=y_pred_train, columns=['VARIABLE_CIBLE']).to_csv(path_or_buf='../y_trained/cont_KNN_5_29.02.26.txt', sep=';')

In [ ]:
y_test = gscv.predict_proba(X_test)[:, 1]
np.savetxt('../subs/continue_KNN5.txt', y_test, fmt='%s')